In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import rustworkx as rx
from rustworkx.visualization import mpl_draw as draw_graph
import numpy as np
from scipy.optimize import minimize
from collections import defaultdict
from typing import Sequence
 
 
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import QAOAAnsatz
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
 
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler

global_print_counter = 0

In [ ]:
### else ###

from qiskit.providers.fake_provider import GenericBackendV2
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from rustworkx.visualization import mpl_draw
import math

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService()
real_backend = service.backend('ibm_strasbourg')


simulator_backend = AerSimulator()

In [ ]:
def build_max_cut_paulis(graph: rx.PyGraph) -> list[tuple[str, float]]:
    """Convert the graph to Pauli list.
 
    This function does the inverse of `build_max_cut_graph`
    """
    pauli_list = []
    for edge in list(graph.edge_list()):
        weight = graph.get_edge_data(edge[0], edge[1])
        pauli_list.append(("ZZ", [edge[0], edge[1]], weight))
    return pauli_list
 


In [ ]:
def cost_func_estimator(params, ansatz, hamiltonian, estimator):
    # transform the observable defined on virtual qubits to
    # an observable defined on all physical qubits
    isa_hamiltonian = hamiltonian.apply_layout(ansatz.layout)
 
    pub = (ansatz, isa_hamiltonian, params)
    job = estimator.run([pub])
    global global_print_counter  
    global_print_counter += 1
 
    results = job.result()[0]
    cost = results.data.evs
 
    return cost

In [ ]:

def optimization(back_end, init_params, candidate_circuit, cost_hamiltonian):
    with Session(backend=back_end) as session:
        # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
        estimator = Estimator(mode=session)
        estimator.options.default_shots = 1000
    
        # Set simple error suppression/mitigation options
        estimator.options.dynamical_decoupling.enable = True
        estimator.options.dynamical_decoupling.sequence_type = "XY4"
        estimator.options.twirling.enable_gates = True
        estimator.options.twirling.num_randomizations = "auto"
    
        result = minimize(
            cost_func_estimator,
            init_params,
            args=(candidate_circuit, cost_hamiltonian, estimator),
            method="COBYLA",
            options={"maxiter": 200, "rhobeg": 1, "catol": 1e-3, "tol": 0.0001},
        )
    return result

In [ ]:
# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
def exam_parameters(session, optimized_circuit):
    sampler = Sampler(mode=session)
    sampler.options.default_shots = 10000
    
    # Set simple error suppression/mitigation options
    sampler.options.dynamical_decoupling.enable = True
    sampler.options.dynamical_decoupling.sequence_type = "XY4"
    sampler.options.twirling.enable_gates = True
    sampler.options.twirling.num_randomizations = "auto"
    
    pub = (optimized_circuit,)
    job = sampler.run([pub], shots=int(1e4))
    global global_print_counter 
    global_print_counter += 1
    counts_int = job.result()[0].data.meas.get_int_counts()
    counts_bin = job.result()[0].data.meas.get_counts()
    shots = sum(counts_int.values())
    final_distribution_int = {key: val / shots for key, val in counts_int.items()}
    final_distribution_bin = {key: val / shots for key, val in counts_bin.items()}
    return (counts_bin, final_distribution_bin, final_distribution_int)

In [ ]:
# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
def exam_parameters_qopt(session, optimized_circuit, shots):
    sampler = Sampler(mode=session)
    sampler.options.default_shots = shots
    
    # Set simple error suppression/mitigation options
    sampler.options.dynamical_decoupling.enable = True
    sampler.options.dynamical_decoupling.sequence_type = "XY4"
    sampler.options.twirling.enable_gates = True
    sampler.options.twirling.num_randomizations = "auto"
    
    pub = (optimized_circuit,)
    job = sampler.run([pub], shots=int(shots))
    global global_print_counter 
    global_print_counter += 1
    job_id = job.job_id()
    print(job_id)
    counts_int = job.result()[0].data.meas.get_int_counts()
    counts_bin = job.result()[0].data.meas.get_counts()
    shots = sum(counts_int.values())
    final_distribution_int = {key: val / shots for key, val in counts_int.items()}
    final_distribution_bin = {key: val / shots for key, val in counts_bin.items()}
    return (counts_bin, final_distribution_bin, final_distribution_int)

In [ ]:
# auxiliary functions to sample most likely bitstring
def find_result_bitstring(final_distribution_int):

    def to_bitstring(integer, num_bits):
        result = np.binary_repr(integer, width=num_bits)
        return [int(digit) for digit in result]
    
    
    keys = list(final_distribution_int.keys())
    values = list(final_distribution_int.values())
    most_likely = keys[np.argmax(np.abs(values))]
    most_likely_bitstring = to_bitstring(most_likely, len(graph))
    return most_likely_bitstring


In [ ]:
def plot_graph_purple(final_distribution_bin, top_n):
    matplotlib.rcParams.update({"font.size": 10})
    final_bits = final_distribution_bin
    values = np.abs(list(final_bits.values()))
    top_4_values = sorted(values, reverse=True)[:top_n]
    positions = []
    for value in top_4_values:
        positions.append(np.where(values == value)[0])
    fig = plt.figure(figsize=(20, 6))
    ax = fig.add_subplot(1, 1, 1)
    plt.xticks(rotation=45)
    plt.title("Result Distribution")
    plt.xlabel("Bitstrings (reversed)")
    plt.ylabel("Probability")
    ax.bar(list(final_bits.keys()), list(final_bits.values()), color="tab:grey")
    for p in positions:
        idx = int(np.array(p).item()) ###
        ax.get_children()[int(p)].set_color("tab:purple")
    plt.show()

In [ ]:
# auxiliary function to plot graphs
def plot_result(G, x):
    colors = ["tab:grey" if i == 0 else "tab:purple" for i in x]
    pos, _default_axes = rx.spring_layout(G), plt.axes(frameon=True)
    rx.visualization.mpl_draw(
        G, node_color=colors, node_size=100, alpha=0.8, pos=pos
    )

In [ ]:
def evaluate_sample(x: Sequence[int], graph: rx.PyGraph) -> float:
    assert len(x) == len(
        list(graph.nodes())
    ), "The length of x must coincide with the number of nodes in the graph."
    return sum(
        w*(x[u] * (1 - x[v]) + x[v] * (1 - x[u]))
        for u, v, w in list(graph.weighted_edge_list())
    )

In [ ]:
def solutions_to_coordinates(solutions):
    arr_d = []
    for i in solutions:
        lst = [int(c) for c in i]
        cut_value_x = evaluate_sample(lst, graph1)
        cut_value_y = evaluate_sample(lst, graph2)
        arr_d.append((cut_value_x, cut_value_y))
    return arr_d

def draw_hv(points, area):
    
    def find_undominated(points):
        undominated = []
        for p in points:
            dominated = False
            for q in points:
                if q[0] >= p[0] and q[1] >= p[1] and q != p:
                    dominated = True
                    break
            if not dominated:
                undominated.append(p)
        return undominated

    undom_points = find_undominated(points)

    # 畫圖
    x_all, y_all = zip(*points)
    x_undom, y_undom = zip(*undom_points)

    plt.title(f'Pareto Front for nodes:{n}, hv:{area:.3f}')
    plt.scatter(x_all, y_all, color='lightgray', label='Dominated points')
    plt.scatter(x_undom, y_undom, color='red', label='Undominated points')
    plt.xlabel('cuts*weights of graph1')
    plt.ylabel('cuts*weights of graph2')
    plt.legend()
    plt.show()


In [ ]:
from scipy.spatial import ConvexHull
# added in the morning
def hv_2d(points):
    points = np.array(points)
    # Add the origin point
    points = np.vstack([points, [0, 0]])
    hull = ConvexHull(points)
    hull_points = points[hull.vertices]

    x = hull_points[:, 0]
    y = hull_points[:, 1]
    area = 0.5 * abs(np.dot(x, np.roll(y, -1)) - np.dot(y, np.roll(x, -1)))
    return area

In [ ]:
area = hv_2d([(0,2), (2,2), (2,0)])
print(area)

# Scale it up #

## preprocessing ##

In [ ]:
### parameters ###
n = 5
initial_gamma = np.pi
initial_beta = np.pi / 2
reps = 4
p1, p2 = 0.5, 0.5
num_dots = 10 ### the amounts of Hc ###


graph = rx.PyGraph()
graph.add_nodes_from(np.arange(0, n, 1))
edge_list = [
    (0, 1, 1),
    (1, 2, 1),
    (2, 3, 1),
    (3, 4, 1),
]
graph.add_edges_from(edge_list)
draw_graph(graph, node_size=600, with_labels=True)
max_cut_paulis = build_max_cut_paulis(graph)
cost_hamiltonian = SparsePauliOp.from_sparse_list(max_cut_paulis, n)

In [ ]:
circuit = QAOAAnsatz(cost_operator=cost_hamiltonian, reps=reps)
circuit.measure_all()
pm = generate_preset_pass_manager(optimization_level=3, backend=real_backend)
candidate_circuit = pm.run(circuit)
init_params = [initial_beta]*reps + [initial_gamma]*reps
result = optimization(simulator_backend, init_params, candidate_circuit, cost_hamiltonian)
print(result)
optimized_params = result.x

## generating graph ##

In [ ]:
n = 18 # number of nodes
generic_backend_ext = GenericBackendV2(n, seed=42)
graph_structure = list(generic_backend_ext.coupling_map)[::2]
num_edges = len(graph_structure)
graph_weight1 = np.random.uniform(1, 10, size=num_edges)
graph_weight2 = (np.max(graph_weight1) + np.min(graph_weight1)) - graph_weight1 \
                   + np.random.uniform(-4, 4, size=num_edges)

In [ ]:
def construct_graph(graph_structure, graph_weight, n):
    graph = rx.PyGraph()
    graph.add_nodes_from(np.arange(0, n, 1))

    edge_list = []
    for (node, weight) in zip(graph_structure, graph_weight):
        edge_list.append((node[0], node[1], weight))

    graph.add_edges_from(edge_list)
    
    return graph
graph1 = construct_graph(graph_structure, graph_weight1, n)
graph2 = construct_graph(graph_structure, graph_weight2, n)
graph_weight = p1*graph_weight1+ p2*graph_weight2
graph = construct_graph(graph_structure, graph_weight, n)

# draw_graph(graph, node_size=600, with_labels=True)
mpl_draw(graph, with_labels=True, edge_labels=str, node_color="#1192E8")

# execution #

In [ ]:
def run_two_hamiltonian(session, graph_structure, graph_weight1, graph_weight2, reps, num_dots, optimized_params):
    ### build graph ###
    
    graph1 = construct_graph(graph_structure, graph_weight1, n)
    graph2 = construct_graph(graph_structure, graph_weight2, n)
    solution_list_total = []
    ### interation start ! ###
    for i in range(num_dots):
        p1, p2 = i/num_dots, 1-i/num_dots
        graph_weight = p1*graph_weight1+ p2*graph_weight2
        graph = construct_graph(graph_structure, graph_weight, n)
        max_cut_paulis = build_max_cut_paulis(graph)
        cost_hamiltonian = SparsePauliOp.from_sparse_list(max_cut_paulis, n)

        ### optimized_circuit ###
        circuit = QAOAAnsatz(cost_operator=cost_hamiltonian, reps=reps)
        circuit.measure_all()
        pm = generate_preset_pass_manager(optimization_level=3, backend=real_backend)
        candidate_circuit = pm.run(circuit)
        model_circuit = candidate_circuit.assign_parameters(optimized_params)

        ### sample ###
        (counts_bin, final_distribution_bin, final_distribution_int) = exam_parameters_qopt(session, model_circuit, 1024)
        
        solution_list = list(counts_bin.keys())
        solution_list_total.extend(solution_list)


    arr_d = solutions_to_coordinates(solution_list_total)
    area = hv_2d(arr_d) #added in the new morning 
    
    ### show result ###
    draw_hv(arr_d, area)
    

with Session(backend=real_backend) as session:
    run_two_hamiltonian(session, graph_structure, graph_weight1, graph_weight2, reps, num_dots, optimized_params)
    